In [1]:
from pymongo import MongoClient
import json
from pprint import pprint
from hashlib import sha1

In [2]:
client = MongoClient('localhost', 27017)
db = client['vacs']
sj = db.sj
hh = db.hh

* `get_data` считывает из файла `.json` информацию по вакансиям, 
* добавляет для каждой вакансии дополнительное значение `_id`, расчитанное как хэш-функция `sha1` от всего набора данных по этой вакансии

In [3]:
def get_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for dct in data:
        dct['_id'] = sha1(str(dct).encode('utf-8')).hexdigest()

    return data

`insert_data` добавляет в указанную коллекцию набор данных (документов)

In [4]:
def insert_data(collection, data):
    collection.insert_many(data)

`find_gt` находит в указанной коллекции все вакансии, у которых предлагаемая зарпалата превышает значение `threshold`

In [5]:
def find_gt(collection, threshold):
    return collection.find({'$or': [
        {'min_salary': {'$gt': threshold}},
        {'max_salary': {'$gt': threshold}}
    ]})

`insert_new_data` добавляет в указанную коллекцию из набора данных `new_data` только те вакансии, которых в этой коллекции еще нет.
Механизм заключается в том, что для одинаковых вакансий будут расчитаны одинаковые `_id`. Путем проверки на свопадение `_id`  добавляемых данных с существующими исключается дублирование данных.

In [6]:
def insert_new_data(collection, new_data):
    exist_id = [_id['_id'] for _id in list(collection.find({}, {'_id': 1}))[:]]
    for dct in new_data:
        if dct['_id'] not in exist_id:
            collection.insert_one(dct)

Очищаем базу данных (для чистоты учебного примера)

In [7]:
MongoClient.drop_database(client, db)

Получаем данные из файлов `.json`. С каждого из сайтов (superjob и hh) по 15 вакансий.

In [8]:
data = {}
data['hh'] = get_data('hh_python_count-15.json')
data['sj'] = get_data('sj_python_count-15.json')
data['sj'][0:2]

[{'name': 'Ведущий разработчик Python / Software development / Team lead',
  'link': 'https://www.superjob.ru/vakansii/veduschij-razrabotchik-python-33224428.html',
  'min_salary': None,
  'max_salary': None,
  'currency': None,
  'source': 'SuperJob.ru',
  '_id': '54e37a565ce294f19e5e4e49b14c81edd9ba2cd4'},
 {'name': 'Программист Python',
  'link': 'https://www.superjob.ru/vakansii/programmist-python-32539725.html',
  'min_salary': None,
  'max_salary': None,
  'currency': None,
  'source': 'SuperJob.ru',
  '_id': '1b255507e9cfcdfea7c7463c504edabc1fb1d478'}]

Вставляем полученные данные в соответствубщие коллекции.

In [9]:
insert_data(hh, data['hh'])
insert_data(sj, data['sj'])

Проверим количество документов в каждой из коллекций.

In [10]:
hh.estimated_document_count(), sj.estimated_document_count()

(15, 15)

Все верно - по 15 вакансий.

Найдем вакансии с сайта hh, у которых предалгаемая зарплата больше 150000

In [11]:
objs = find_gt(hh, 150000)
for obj in objs:
    pprint(obj)
    print()

{'_id': 'b58713616f9964c4605c8acac152121c98210e23',
 'currency': 'RUR',
 'link': 'https://sergievposad.hh.ru/vacancy/35011897?query=python',
 'max_salary': None,
 'min_salary': 200000.0,
 'name': 'Data Scientist',
 'source': 'hh.ru'}

{'_id': 'bfd336cdcead100daaf1e01c750ad79b6fa3f6a1',
 'currency': 'RUR',
 'link': 'https://sergievposad.hh.ru/vacancy/34746892?query=python',
 'max_salary': None,
 'min_salary': 230000.0,
 'name': 'Senior Frontend Developer (JavaScript, React)',
 'source': 'hh.ru'}

{'_id': 'a0513a3add12bb0cdcb5883871fa43deb632cc66',
 'currency': 'RUR',
 'link': 'https://sergievposad.hh.ru/vacancy/34977652?query=python',
 'max_salary': 300000.0,
 'min_salary': 130000.0,
 'name': 'Python Developer',
 'source': 'hh.ru'}

{'_id': 'cb60e40597c0bc25e90ca5cc3dfdb16ab326b0b7',
 'currency': 'RUR',
 'link': 'https://sergievposad.hh.ru/vacancy/34850179?query=python',
 'max_salary': 200000.0,
 'min_salary': 120000.0,
 'name': 'Backend python/Django разработчик',
 'source': 'hh.ru'}



Найдено 5 вакансий.


Найдем вакансии с сайта hh, у которых предалгаемая зарплата больше 80000

In [12]:
objs = find_gt(sj, 80000)
for obj in objs:
    pprint(obj)
    print()

{'_id': 'b7ded45e1a708b24ee8d0c7fda1bd3d5d06fb33d',
 'currency': 'RUB',
 'link': 'https://www.superjob.ru/vakansii/web-razrabotchik-33214670.html',
 'max_salary': 100000.0,
 'min_salary': 80000.0,
 'name': 'Web-разработчик / Web-программист Yii2/Python/Python '
         'Django/Django/Laravel/Vue',
 'source': 'SuperJob.ru'}

{'_id': 'ee8ec548794265c76400145bede9e0f688c20562',
 'currency': 'RUR',
 'link': 'https://www.superjob.ru/vakansii/analitik-dannyh-33270372.html',
 'max_salary': 150000.0,
 'min_salary': None,
 'name': 'Аналитик данных (Data analyst)',
 'source': 'SuperJob.ru'}

{'_id': 'ed0bb316d1ad2f015a7ec9d55866909c83d08e25',
 'currency': 'RUR',
 'link': 'https://www.superjob.ru/vakansii/sistemnyj-analitik-32663136.html',
 'max_salary': None,
 'min_salary': 81000.0,
 'name': 'Системный аналитик / Ведущий системный администратор',
 'source': 'SuperJob.ru'}



Найдено 3 вакансии.

Получим новые данные из фалйов `.json`. Теперь у нас будут наборы по 20 вакансий от каждого сайта, причем новых вакансий в каждом наборе будет всего по 5 штук.

In [13]:
new_data = {}
new_data['hh'] = get_data('hh_python_count-20.json')
new_data['sj'] = get_data('sj_python_count-20.json')

Используя проверку по `_id`, расчитанному с помощью `sha1`, добавим в наши коллекции только новые вакансии, отсеяв уже имеющиеся

In [14]:
insert_new_data(hh, new_data['hh'])
insert_new_data(sj, new_data['sj'])

Проверим, как изменилось количество документов в коллекциях.

In [15]:
hh.estimated_document_count(), sj.estimated_document_count()

(20, 20)

Все верно - добавилось по 5 новых вакансий в каждую коллекцию.